### Для классификации тональности русскоязычных отзывов о застройщиках  будем использовать **предобученную языковую модель ruBERT-large с Hugging Face от SberDevices**

---



### **План обучения выглядит так:**

1.   Загрузка и подготовка данных (определение целевой переменной, разметка данных)
2.   Предобработка текста
3.   Настройка модели
4.   Подбор гиперпараметров через кросс-валидацию с Optuna
5.   Оценка качетсва модели на размеченных данных





### **1. Загрузка и подготовка данных**

Для определения целевой переменной (тональности) воспользуемся рейтингом отзыва - поле ***rating***. Поле ***rating*** принимает значения от **1** до **5** (звездочки в отзыве). Мы переведем их в метки тональности: отзывы с рейтингом **1** или **2** считаются **негативными** (*-1*, закодируем как класс *0*), рейтинг **3** — **нейтральный** (*0*, класс *1*), а **4** или **5** — **позитивный** (*+1*, класс *2*).

In [ ]:
import pandas as pd

df = pd.read_csv('all_reviews_bert.csv')
df

In [ ]:
# Map ratings (1-5) to sentiment labels (0=negative, 1=neutral, 2=positive)
def rating_to_label(rating):
    if rating <= 2.0:
        return 0
    elif rating == 3.0:
        return 1
    else:
        return 2

df['sentiment_label'] = df['rating'].apply(rating_to_label).astype(int)

In [ ]:
# Sentiment class distribution
class_counts = df['sentiment_label'].value_counts()
print("Class distribution:")
for label, count in class_counts.items():
    print(f"{label}: {count}")

Видим, что классы распределены неравномерно - наблюдается явный дисбаланс.

Для надёжной оценки качества модели будем использовать стратифицированную разбивку данных на тренировочную и тестовую выборки, сохраняя пропорции классов. Отложим **20% данных** в качестве тестовой выборки (holdout) для финальной оценки, а оставшиеся 80% используем для обучения и подбора гиперпараметров.


In [ ]:
from sklearn.model_selection import train_test_split

# Splitting dataset on training and test sample (startified by sentiment)
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    stratify=df['sentiment_label'],
    random_state=42)

print(f"Train sample size: {len(train_df)}, Test sample size: {len(test_df)}")

# Reseting index
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

### **2. Предобработка текста с помощью Natasha**

Перед обучением модели выполним предобработку текста: очистим от лишних символов и приведем слова к нормальной форме (лемматизация). Для русского языка мы используем библиотеку **Natasha**, которая обеспечивает качественную токенизацию и лемматизацию с учетом морфологии.

Шаги предобработки для каждого отзыва:
*   Токенизация текста (разбиение на слова и знаки препинания)
*   Удаление или игнорирование пунктуации, чисел и лишних символов
*   Приведение слов к начальной форме (лемматизация) с помощью морфологического анализатора Natasha
*   Приведение текста к нижнему регистру


После этих шагов текст станет более стандартизированным, что облегчает обучение модели (модель не будет учитывать различия в формах слов, а лишь их основы). Ниже определим функцию **preprocess_text** для этой задачи. Затем применим её ко всем отзывам в обучающей и тестовых выборках.

In [ ]:
!pip install natasha

In [ ]:
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab, Doc
import re

segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

def lemmatize_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^а-яёА-ЯЁa-zA-Z\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text)

    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)

    lemmas = " ".join([token.lemma for token in doc.tokens if token.lemma])
    return lemmas

train_df['text_processed'] = train_df['full_review_text'].apply(lemmatize_text)
test_df['text_processed']  = test_df['full_review_text'].apply(lemmatize_text)

Пример: посмотрим оригинальный и обработанный текст для пары отзывов

In [ ]:
example_indices = [0, 1]
for idx in example_indices:
    orig_text = train_df.loc[idx, 'full_review_text']
    proc_text = train_df.loc[idx, 'text_processed']
    print(f"Example {idx}:")
    print("Original text:", orig_text)
    print("After lemmatization:", proc_text)

### 3. **Токенизация и формирование наборов данных для модели BERT**

Для обучения модели BERT нам нужно преобразовать тексты в числовой формат – токенизировать их с помощью соответствующего токенизатора. Используем предобученный токенизатор от модели ai-forever/ruBert-large из библиотеки 🤗 Transformers. Он приведет каждое слово (или часть слова) к идентификатору из словаря модели.

- Зададим максимальную длину последовательности MAX_LEN = 256 токенов. Если отзыв длиннее, он будет усечен до 256 токенов, если короче – будет дополнен специальными токенами заполнителями при формировании батча.
- Чтобы эффективно обрабатывать данные разной длины, воспользуемся DataCollatorWithPadding. Он будет автоматически паддировать (дополнять) меньшие последовательности до длины самой длинной в текущем батче. Так мы не расходуем лишнюю память на глобальное дополнение всех последовательностей до 256, а только внутри батча.
- Подготовим кастомный класс Dataset для наших данных, который по индексу будет возвращать токенизированный отзыв и соответствующую метку. Это позволит легко использовать DataLoader для итерации по данным батчами.

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import Dataset

# Initializing ruBERT-large tokenizator
model_name = "ai-forever/ruBert-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
token_max_len = 256

# Tokenizing training and test sample texts
train_encodings = tokenizer(
    list(train_df['text_processed']),
    truncation=True,
    padding=False,
    max_length=token_max_len
)

test_encodings = tokenizer(
    list(test_df['text_processed']),
    truncation=True,
    padding=False,
    max_length=token_max_len
)

# Prepare torch Dataset objects
class ReviewDataset(Dataset):
    def __init__(self, encodings, labels):
        # vocabulary with keys 'input_ids', 'attention_mask'
        self.encodings = encodings
        # labels list
        self.labels = labels
    def __getitem__(self, idx):
        # Taking tokenized representations for this index
        item = {key: val[idx] for key, val in self.encodings.items()}
        # Adding labels
        item['labels'] = self.labels[idx]
        return item
    def __len__(self):
        return len(self.labels)

# Creating Dataset objects for training and test samples
train_dataset = ReviewDataset(train_encodings, train_df['sentiment_label'].tolist())
test_dataset = ReviewDataset(test_encodings, test_df['sentiment_label'].tolist())

# Creating collator for dynamic padding batches
collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

Пример: продемонстрируем работу токенизатора на обработанном тексте

In [ ]:
sample_text = train_df.loc[0, 'text_processed']
tokens = tokenizer.tokenize(sample_text)
input_ids = train_encodings['input_ids'][0]

print("Lemmatized text:")
print(sample_text)
print("\nBERT tokens:")
print(tokens)
print("\nTokens IDs:")
print(input_ids)

### **4. Настройка обучения: модель, оптимизатор и гиперпараметры**

Мы используем предобученную модель **BertForSequenceClassification** (на основе ruBERT-large) с выходным слоем классификации на 3 класса. Модель добавляет над [CLS]-представлением линейный классификатор и softmax для предсказания вероятностей классов.


---


#### **План обучения:**

- **Модель:** BertForSequenceClassification.from_pretrained('ai-forever/ruBert-large', num_labels=3).
- **Оптимизатор:** AdamW (адам с декей на весах) – стандарт для BERT. Мы настроим weight decay для всех весов кроме коэффициентов смещения и LayerNorm (их обычно не подвергают L2-регуляризации).
- **Scheduler:** линейный планировщик обучения с прогревом (get_linear_schedule_with_warmup) – сначала небольшое warmup_ratio эпох итераций для разогрева (низкий learning rate), затем линейное уменьшение learning rate до 0 к концу обучения.
- **Loss-функция:** будем экспериментировать с обычной кросс-энтропией (CrossEntropyLoss) и Focal Loss, которая может улучшить обучение на несбалансированных данных, фокусируясь на трудноклассифицируемых примерах. Также рассмотрим два подхода к учёту дисбаланса:
    - Взвешивание классов (class weights) в функции потерь.
    - WeightedRandomSampler – балансировка выборки путём более частого выбора редких классов при формировании батчей.



---



#### **Нам предстоит подобрать ряд гиперпараметров:**
- ***learning_rate*** – скорость обучения (от 1e-5 до 5e-5, логарифмический масштаб).
- ***batch_size*** – размер батча (8, 16 или 32).
- ***weight_decay*** – коэффициент L2-регуляризации (0.0 до 0.1).
- ***epochs*** – максимальное число эпох обучения (от 2 до 6).
- ***warmup_ratio*** – доля разогрева (0.0, 0.1 или 0.2 от всего числа шагов).
- ***loss_fn*** – тип функции потерь: ce (CrossEntropy) или focal (FocalLoss).
- ***balance_strategy*** – стратегия борьбы с дисбалансом: class_weights или sampler.

Чтобы найти оптимальные значения этих гиперпараметров, мы применим поиск с помощью **Optuna**. В качестве целевой метрики будем максимизировать **Macro F1** на валидации. Мы разделим обучающие данные на 5 фолдов (Stratified K-Fold) и будем в рамках каждого набора гиперпараметров обучать модель на 4 фолдах и оценивать Macro F1 на оставшемся фолде, усредняя результаты по 5 фолдам. Таким образом, Optuna будет оценивать качество каждого набора гиперпараметров на основе кросс-валидации

Во время обучения на каждом фолде реализуем Early Stopping: если Macro F1 на валидации не улучшается в течение 2 эпох подряд, останавливаем обучение на этом фолде досрочно, чтобы не переобучаться и сэкономить время. Будем сохранять наилучшее значение F1 для каждого фолда.



---



#### **Настройка Optuna и кросс-валидации**

Определим функцию-цель для Optuna, которая принимает набор гиперпараметров, обучает модель на 5-фолдовой кросс-валидации и возвращает средний Macro F1. Внутри нее происходит следующее:

- Инициализация модели и оптимизатора для каждого фолда с заданными гиперпараметрами.
- Применение выбранной стратегии балансировки: либо установка весов классов в функции потерь, либо использование WeightedRandomSampler для генерации батчей.
- Вычисление количества тренировочных шагов и настройка linear scheduler с разогревом.
- Цикл обучения по эпохам с отслеживанием best F1 на валидации и условием ранней остановки.
- После обучения на всех фолдах – возврат среднего Macro F1 для данного набора гиперпараметров.

In [ ]:
!pip install transformers

In [ ]:
!pip install accelerate

In [ ]:
!pip install optuna

In [ ]:
import torch, numpy as np, optuna
from torch import nn
from torch.optim import AdamW
from torch.utils.data import WeightedRandomSampler
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from accelerate import Accelerator
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 1. Focal-loss
def focal_loss(logits, targets, gamma=2.0, class_weights=None):
    log_probs = nn.functional.log_softmax(logits.float(), dim=1)
    probs     = torch.exp(log_probs)
    pt        = probs[range(len(targets)), targets]
    log_pt    = log_probs[range(len(targets)), targets]
    alpha     = class_weights[targets] if class_weights is not None else torch.ones_like(pt)
    return (-alpha * ((1 - pt) ** gamma) * log_pt).mean()

# 2. Stratified 5-fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# array of labels
y_all = train_df['sentiment_label'].values

# 3. Optuna objective
def objective(trial):
    learning_rate  = trial.suggest_float("learning_rate", 1e-5, 5e-5, log=True)
    batch_size     = trial.suggest_categorical("batch_size", [8, 16, 32])
    weight_decay   = trial.suggest_float("weight_decay", 0.0, 0.1)
    epochs         = trial.suggest_int("epochs", 2, 6)
    warmup_ratio   = trial.suggest_categorical("warmup_ratio", [0.0, 0.1, 0.2])
    loss_fn_name   = trial.suggest_categorical("loss_fn", ["ce", "focal"])
    balance_strat  = trial.suggest_categorical("balance_strategy", ["class_weights", "sampler"])

    total_f1 = 0.0

    for fold, (train_idx, val_idx) in enumerate(skf.split(np.zeros(len(y_all)), y_all), 1):

        print(f"Fold {fold}/5  —  trial #{trial.number}")
        train_subset = torch.utils.data.Subset(train_dataset, train_idx)
        val_subset   = torch.utils.data.Subset(train_dataset,   val_idx)

        # Balancing classes
        if balance_strat == "sampler":
            class_cnt  = np.bincount(y_all[train_idx], minlength=3)
            w_cls      = 1.0 / class_cnt
            sample_w   = [w_cls[y] for y in y_all[train_idx]]
            sampler    = WeightedRandomSampler(sample_w, len(sample_w), replacement=True)
            train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size,
                                                       sampler=sampler, collate_fn=collator)
        else:
            train_loader = torch.utils.data.DataLoader(train_subset, batch_size=batch_size,
                                                       shuffle=True,  collate_fn=collator)

        val_loader   = torch.utils.data.DataLoader(val_subset,   batch_size=batch_size,
                                                   shuffle=False, collate_fn=collator)

        # Model and optimizator
        model = BertForSequenceClassification.from_pretrained(
            'ai-forever/ruBert-large',
            num_labels=3
        )

        no_decay = ["bias", "LayerNorm.weight"]
        param_groups = [
            {"params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
             "weight_decay": weight_decay},
            {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
             "weight_decay": 0.0}
        ]

        optimizer = AdamW(param_groups, lr=learning_rate)

        steps_per_epoch   = len(train_loader)
        total_train_steps = epochs * steps_per_epoch
        warmup_steps      = int(warmup_ratio * total_train_steps)
        scheduler = get_linear_schedule_with_warmup(optimizer, warmup_steps, total_train_steps)

        accelerator = Accelerator(mixed_precision="fp16")
        model, optimizer, train_loader, val_loader, scheduler = accelerator.prepare(
            model, optimizer, train_loader, val_loader, scheduler
        )

        # loss-function
        if loss_fn_name == "ce":
            if balance_strat == "class_weights":
                cnt = np.bincount(y_all[train_idx], minlength=3)
                w   = torch.tensor((len(train_idx)/cnt)/3.0, dtype=torch.float).to(accelerator.device)
                criterion = nn.CrossEntropyLoss(weight=w)
            else:
                criterion = nn.CrossEntropyLoss()
        else:
            cnt = np.bincount(y_all[train_idx], minlength=3) if balance_strat=="class_weights" else None
            class_w = torch.tensor((len(train_idx)/cnt)/3.0, dtype=torch.float).to(accelerator.device) if cnt is not None else None
            criterion = None

        # Epochs cycle with progress bar
        best_f1_fold, epochs_no_improve = 0.0, 0
        epoch_bar = tqdm(range(1, epochs+1), desc=f"Epochs (fold {fold})", leave=False)

        for epoch in epoch_bar:
            model.train()
            batch_bar = tqdm(train_loader, desc=f"Train ep{epoch}", leave=False)

            total_train_loss = 0.0
            for batch in batch_bar:
                optimizer.zero_grad()
                outputs = model(input_ids=batch['input_ids'],
                                attention_mask=batch['attention_mask'],
                                token_type_ids=batch.get('token_type_ids', None))
                logits = outputs.logits

                if loss_fn_name == "ce":
                    loss = criterion(logits, batch['labels'])
                else:
                    loss = focal_loss(logits, batch['labels'], gamma=2.0, class_weights=class_w)

                accelerator.backward(loss)
                optimizer.step()
                scheduler.step()

                total_train_loss += loss.item()
                batch_bar.set_postfix(loss=float(loss))

            # Validation
            model.eval()
            all_preds, all_labels = [], []
            for batch in val_loader:
                with torch.no_grad():
                    logits = model(input_ids=batch['input_ids'],
                                   attention_mask=batch['attention_mask'],
                                   token_type_ids=batch.get('token_type_ids', None)).logits
                    preds  = torch.argmax(logits, dim=1)
                all_preds.extend(accelerator.gather(preds).cpu().numpy().tolist())
                all_labels.extend(accelerator.gather(batch['labels']).cpu().numpy().tolist())

            val_f1 = f1_score(all_labels, all_preds, average='macro')
            print(f"📈 Fold {fold}  Epoch {epoch}: F1={val_f1:.4f}")
            epoch_bar.set_postfix(F1=val_f1)

            if val_f1 > best_f1_fold:
                best_f1_fold, epochs_no_improve = val_f1, 0
            else:
                epochs_no_improve += 1
            if epochs_no_improve >= 2:
                break

        print(f"🟩 Fold {fold}  Best F1: {best_f1_fold:.4f}")
        total_f1 += best_f1_fold

        accelerator.free_memory()
        torch.cuda.empty_cache()

    # Average Macro-F1 on 5 folds
    return total_f1 / 5.0

# Optuna
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20, show_progress_bar=True)

print("Best hyperparameters:", study.best_trial.params)
print(f"Best average F1 on validation: {study.best_value:.4f}")

In [ ]:
import torch, numpy as np, optuna
from torch import nn
from torch.optim import AdamW
from torch.utils.data import WeightedRandomSampler
from transformers import BertForSequenceClassification, get_linear_schedule_with_warmup
from accelerate import Accelerator
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold
from tqdm.auto import tqdm

def focal_loss(logits, targets, gamma=2.0, class_weights=None):
    log_probs = nn.functional.log_softmax(logits.float(), dim=1)
    probs = torch.exp(log_probs)
    pt = probs[range(len(targets)), targets]
    log_pt = log_probs[range(len(targets)), targets]
    if class_weights is not None:
        alpha = class_weights[targets]
    else:
        alpha = torch.ones_like(pt)
    loss = -alpha * ((1 - pt) ** gamma) * log_pt
    return loss.mean()

best_lr = 2.980830878730812e-05
best_batch = 16
best_weight_decay = 0.07233960382914938
best_epochs = 5
best_warmup_ratio = 0.1
best_loss_fn = 'focal'
best_balance = 'sampler'

# Обучение финальной модели на всей обучающей выборке с лучшими гиперпараметрами
final_model = BertForSequenceClassification.from_pretrained('ai-forever/ruBert-large', num_labels=3)

# Настройка оптимизатора и планировщика для финальной модели
no_decay = ["bias", "LayerNorm.weight"]
param_groups = [
    {"params": [p for n, p in final_model.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": best_weight_decay},
    {"params": [p for n, p in final_model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
]
optimizer = AdamW(param_groups, lr=best_lr)

# Даталоадер для полной обучающей выборки
if best_balance == "sampler":
    # WeightedRandomSampler на всей обучающей выборке
    full_train_labels = train_df['sentiment_label'].values
    class_counts = np.bincount(full_train_labels, minlength=3)
    class_weights_for_sampler = 1.0 / class_counts
    sample_weights = [class_weights_for_sampler[label] for label in full_train_labels]
    sampler = torch.utils.data.WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
    full_train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=best_batch, sampler=sampler, collate_fn=collator)
else:
    full_train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=best_batch, shuffle=True, collate_fn=collator)

# Считаем шаги и настраиваем scheduler
num_steps_per_epoch = len(full_train_loader)
total_steps = best_epochs * num_steps_per_epoch
warmup_steps = int(best_warmup_ratio * total_steps)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# Accelerator для финального обучения
accelerator = Accelerator(mixed_precision="fp16")
final_model, optimizer, full_train_loader, scheduler = accelerator.prepare(final_model, optimizer, full_train_loader, scheduler)

# Настройка функции потерь для финального обучения
if best_loss_fn == "ce":
    if best_balance == "class_weights":
        class_counts = np.bincount(train_df['label'].values, minlength=3)
        class_weights = (len(train_df) / class_counts) / 3.0
        class_weights = torch.tensor(class_weights, dtype=torch.float).to(accelerator.device)
        final_criterion = nn.CrossEntropyLoss(weight=class_weights)
    else:
        final_criterion = nn.CrossEntropyLoss()
else:  # focal
    if best_balance == "class_weights":
        class_counts = np.bincount(train_df['label'].values, minlength=3)
        class_weights = (len(train_df) / class_counts) / 3.0
        class_weights = torch.tensor(class_weights, dtype=torch.float).to(accelerator.device)
    else:
        class_weights = None
    final_criterion = None  # будем вызывать focal_loss вручную

final_model.train()
for epoch in tqdm(range(1, best_epochs + 1), desc="Эпохи"):
    epoch_loss = 0.0
    batch_bar = tqdm(full_train_loader, desc=f"Эпоха {epoch}", leave=False)

    for batch in batch_bar:
        optimizer.zero_grad()
        outputs = final_model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch.get('token_type_ids', None),
            labels=None
        )
        logits = outputs.logits

        if best_loss_fn == "ce":
            loss = final_criterion(logits, batch['labels'])
        else:
            loss = focal_loss(logits, batch['labels'], gamma=2.0, class_weights=class_weights)

        accelerator.backward(loss)
        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item()
        batch_bar.set_postfix(loss=float(loss))

    tqdm.write(f"📊 Эпоха {epoch}/{best_epochs} — Средний loss: {epoch_loss/len(full_train_loader):.4f}")

# Отключаем режим обучения перед оценкой
final_model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
final_model.to(device)

# Оценка на тестовой (holdout) выборке
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=best_batch, shuffle=False, collate_fn=collator)

# all_preds = []
# all_probs = []
# all_true = []

# for batch in test_loader:
#     with torch.no_grad():
#         outputs = final_model(input_ids=batch['input_ids'], attention_mask=batch['attention_mask'], token_type_ids=batch.get('token_type_ids', None))
#         logits = outputs.logits
#         probs = nn.functional.softmax(logits, dim=1)
#         preds = torch.argmax(probs, dim=1)
#     # Собираем результаты (на CPU для вычисления метрик)
#     all_preds.extend(preds.cpu().numpy().tolist())
#     all_probs.extend(probs.cpu().numpy().tolist())
#     all_true.extend(batch['labels'].cpu().numpy().tolist())

all_preds = []
all_probs = []
all_true = []

for batch in test_loader:
    # ВАЖНО: переносим входной батч на устройство модели
    batch = {k: v.to(device) for k, v in batch.items()}

    with torch.no_grad():
        outputs = final_model(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch.get('token_type_ids', None)
        )
        logits = outputs.logits
        probs = nn.functional.softmax(logits, dim=1)
        preds = torch.argmax(probs, dim=1)

    # ПЕРЕНОС НА CPU ДЛЯ sklearn
    all_preds.extend(preds.cpu().numpy().tolist())
    all_probs.extend(probs.cpu().numpy().tolist())
    all_true.extend(batch['labels'].cpu().numpy().tolist())


# Преобразуем в numpy массивы
all_preds = np.array(all_preds)
all_probs = np.array(all_probs)
all_true = np.array(all_true)

# Вычисление метрик
accuracy = accuracy_score(all_true, all_preds)
macro_f1 = f1_score(all_true, all_preds, average='macro')
macro_roc_auc = roc_auc_score(all_true, all_probs, average='macro', multi_class='ovr')
cm = confusion_matrix(all_true, all_preds)

print(f"\nМетрики на тестовой выборке:")
print(f"Accuracy = {accuracy:.4f}")
print(f"Macro F1 = {macro_f1:.4f}")
print(f"Macro ROC-AUC = {macro_roc_auc:.4f}")

# Вывод confusion matrix
labels_name = {0: "neg", 1: "neu", 2: "pos"}
cm_df = pd.DataFrame(cm, index=[labels_name[i] for i in range(3)], columns=[labels_name[i] for i in range(3)])
print("\nConfusion Matrix (actual x predicted):")
print(cm_df)

### **Анализ отзывов: тематическая модель с HDBSCAN и c-TF-IDF**

---

В этом разделе мы проведём тематическое моделирование пользовательских отзывов о жилых комплексах. Для этого мы используем эмбеддинги предложений (Sentence Transformers) для представления отзывов в векторном виде, метод снижения размерности UMAP для визуализации и кластеризацию HDBSCAN для автоматического выделения групп схожих отзывов. После кластеризации для каждой группы (темы) вычислим ключевые слова с помощью c-TF-IDF (class-based TF-IDF), что позволяет определить, какие слова наиболее характерны для данной темы. Смотреть отзывы будем в разрезе отрицательных (sentiment_label = 0), а также неотрицательных (sentiment_label = 1/2)

#### **1. Загрузка и подготовка данных**

In [ ]:
import pandas as pd

df = pd.read_csv('reviews_with_sentiment.csv')
print("Всего отзывов:", len(df))
print("Столбцы:", df.columns.tolist())
print("Пример записей:")
print(df[['doc_id', 'developer', 'rating', 'full_review_text', 'sentiment_label']])

Каждая запись содержит **текст отзыва (full_review_text)**, **идентификатор документа (doc_id)**, **имя застройщика (developer)**, **оценку (rating)** и **метку тональности sentiment_label (где 0 – негативный отзыв, 1 – нейтральный, 2 – позитивный)**. Ниже разделим датасет на две части по тональности:

In [ ]:
negative_reviews = df[df['sentiment_label'] == 0]
non_negative_reviews = df[df['sentiment_label'].isin([1, 2])]

print("Отрицательных отзывов:", len(negative_reviews))
print("Неотрицательных отзывов:", len(non_negative_reviews))

#### **2. Векторизация отзывов моделью Sentence Transformer**

Для тематического анализа преобразуем тексты отзывов в эмбеддинги – численные вектора, отражающие семантическое содержание текста. Мы используем предобученную многоязычную **модель SentenceTransformer**, способную превращать фразы на русском языке в векторное представление.

In [ ]:
!pip install -q sentence-transformers umap-learn hdbscan

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

Теперь получим эмбеддинги для отзывов. Чтобы избежать повторных вычислений, сформируем сразу векторные представления для всех отзывов

In [ ]:
all_reviews = df['full_review_text'].astype(str).tolist()
embeddings = model.encode(all_reviews, show_progress_bar=True)

Вот как выглядит отзыв после преобразования в вектор (эмбеддинг)

In [ ]:
print("Размерность эмбеддинга одного отзыва:", embeddings.shape[1])
print("Отзыв:\n", all_reviews[0])
print("Эмбеддинг отзыва:\n", embeddings[0][:5], "...")

Модель преобразовала каждый отзыв в 384-мерный вектор. Эти эмбеддинги будут основой для кластеризации – отзывы с похожим содержанием должны иметь схожие векторы.

#### **3. Кластеризация отзывов HDBSCAN/K-means+ выделение тем через c-TF-IDF**

Для кластеризации используем алгоритм HDBSCAN (Hierarchical DBSCAN), который автоматически определяет плотные кластеры разного размера и помечает точки, не принадлежащие ни одному кластеру, как шум (outliers). Важный параметр – min_cluster_size=15, то есть тема должна содержать минимум 15 отзывов, иначе отзывы считаются выбросами. Это позволяет отсеять единичные уникальные отзывы, оставляя только повторяющиеся темы.

После получения кластеров определим темы – т.е. интерпретацию кластеров – с помощью c-TF-IDF. Этот метод комбинирует все тексты в кластере в один "документ" и вычисляет TF-IDF для этого агрегированного документа относительно всех кластеров, Таким образом, получаем слова, наиболее характерные для данного кластера (частые в нём и редкие в других кластерах)а затем находим их веса и извлекаем топ-10 важных слов каждого кластера.